In [1]:
amazon <- read.csv('amz_com-ecommerce_sample.csv' ,encoding="ANSI_X3.4-1968",header = T)
flipkart <- read.csv('flipkart_com-ecommerce_sample.csv',encoding="ANSI_X3.4-1968",header = T)

## Checking which columns have mismatches

In [2]:
all.equal(amazon,flipkart) |> t() 

Component “product_name”: 3445 string mismatches,Component “product_category_tree”: 1 string mismatch,Component “retail_price”: 'is.NA' value mismatch: 78 in current 0 in target,Component “discounted_price”: 'is.NA' value mismatch: 78 in current 0 in target,Component “description”: 5 string mismatches,Component “product_specifications”: 1 string mismatch


### The **description** category has the most mismatches, we will use that for matching.    
The description is also more likely to be unique.



In [3]:
selectcol <- 'description'
(index <- which(amazon[,selectcol]!=flipkart[,selectcol]) )


[1]  29  67 391 485 486

## The below function will match names using *cosine*  distance after detecting mismatch in the selected column

In [4]:
match_by_selected_column<- function(selectcol,amazon,flipkart){
(index <- which(amazon[,selectcol]!=flipkart[,selectcol]) )



mismatch <- cbind(  amazon[index,c('product_name',selectcol)],
                    flipkart[index,c('product_name',selectcol)]
                  )

colnames(mismatch)[3:4] <- paste0('flipkart_',colnames(mismatch)[3:4]) 
colnames(mismatch)[1:2] <- paste0('amazon_',colnames(mismatch)[1:2]) 
print("mismatched")

library(stringdist)

stringdistmatrix(
                  tolower(mismatch$flipkart_product_name),
                  tolower(mismatch$amazon_product_name),
                  method = "cosine"
                 )  |> apply(2,which.min) -> temp

return(data.frame(mismatch$amazon_product_name[temp],mismatch$flipkart_product_name))
}

In [5]:
matched<-match_by_selected_column(selectcol = selectcol ,amazon=amazon,flipkart = flipkart)

[1] "mismatched"


In [6]:
head(matched)

,mismatch.amazon_product_name.temp.,mismatch.flipkart_product_name
,<chr>,<chr>
1,FDT WOMEN'S Leggings Pants,FDT Women's Leggings
2,MASARA SOLID WOMEN'S STRAIGHT KURTA,MASARA Solid Women's Straight Kurta
3,Fantasy Ika Casual Short Sleeve Solid Women's Top,Fantasy Ika Casual Short Sleeve Solid Women's Top
4,"SFDS Casual, Formal, Party Sleeveless Self Design Women's Top","SFDS Casual, Formal, Party Sleeveless Self Design Women's Top"
5,"SFDS Casual, Formal, Party Sleeveless Self Design Women's Top","SFDS Casual, Formal, Party Sleeveless Self Design Women's Top"


In [7]:
fixed_amazon <- amazon
rm(amazon)
fixed_amazon[index,'product_name'] <- matched[,1]

## Combined all


In [8]:
combined <- data.frame(cbind(
              n1=  flipkart[,'product_name'],
              r1=  flipkart[,'retail_price'],
              d1=  flipkart[,'discounted_price'],
              n2=  fixed_amazon[,'product_name'],
              r2=  fixed_amazon[,'retail_price'],
              d2=  fixed_amazon[,'discounted_price']
                          ))

coln <-c('Product name in Flipkart','Retail price in Flipkart','Discounted Price in Flipkart','Product name in Amazon','Retail price in Amazon','Discounted Price in Amazon')
colnames(combined) <- coln
head(combined)

,Product name in Flipkart,Retail price in Flipkart,Discounted Price in Flipkart,Product name in Amazon,Retail price in Amazon,Discounted Price in Amazon
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Alisha Solid Women's Cycling Shorts,999,379,Alisha Solid Women's Cycling Shorts,982,438
2,FabHomeDecor Fabric Double Sofa Bed,32157,22646,FabHomeDecor Fabric Double Sofa Bed,32143,29121
3,AW Bellies,999,499,AW Bellies,991,551
4,Alisha Solid Women's Cycling Shorts,699,267,Alisha Solid Women's Cycling Shorts,694,325
5,Sicons All Purpose Arnica Dog Shampoo,220,210,Sicons All Purpose Arnica Dog Shampoo,208,258
6,Eternal Gandhi Super Series Crystal Paper Weights with Silver Finish,430,430,Eternal Gandhi Super Series Crystal Paper Weights with Silver Finish,427,473


In [9]:
user_input <- function(input){
  
  if(input %in% combined$`Product name in Flipkart` | input %in% combined$`Product name in Amazon`)
  {
      
    matches <- union(
                     match(input,combined$`Product name in Flipkart`),
                     match(input,combined$`Product name in Amazon`) 
                    )
    }
    return(combined[matches,])
    }
    

In [10]:
user_input('AW Bellies')


,Product name in Flipkart,Retail price in Flipkart,Discounted Price in Flipkart,Product name in Amazon,Retail price in Amazon,Discounted Price in Amazon
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
3,AW Bellies,999,499,AW Bellies,991,551
